<a href="https://colab.research.google.com/github/vkrisvasan/GeminiAI_KV/blob/main/kv_fashion_buddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-cloud-aiplatform ragstack-ai --upgrade


In [71]:
!pip install zlib-ng

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 1.1 MB/s eta 0:00:00


In [72]:
import pandas as pd
import numpy as np
import getpass, os, requests, sys, json, vertexai
import zlib

In [6]:
credential_names = ["GCP_PROJECT_ID", "ASTRA_DB_ENDPOINT", "ASTRA_DB_TOKEN","HF_TOKEN"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)
print(os.environ["GCP_PROJECT_ID"])

Provide your...HF_TOKEN··········
sustained-tree-423806-c6


In [4]:
from google.colab import userdata, auth, files, drive
from google.cloud import aiplatform
print(os.environ["GCP_PROJECT_ID"])

!gcloud config set project {os.environ["GCP_PROJECT_ID"]}
auth.authenticate_user()


sustained-tree-423806-c6
Updated property [core/project].


In [79]:
from datasets import load_dataset
dataset = load_dataset("vkrisvasan/zara_embedding",split='train')


In [ ]:
# How create embedding by creating a embedding model
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
(x)
vertexai. init(project=os.getenv("GCP_PROJECT_ID"))
model = NultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
Generate and add the embeddings to the dataframe
def. add_embeddings(df):
for i in range(len(df)):
if pd.isnull(df.loc[i, 'embeddings]).all():
◦ Create Image from URL
img_bytes = requests.get(df.loc[i, "product_imağes"]).content
img = Image(img_bytes)
# Create embedding based on Image
cur_embeddings = model.get_embeddings(
image= img,
contextual_text = df.loc[i, 'product_name'])
df.locli, 'embeddings'] = cur_embeddings.image_embedding
add_embeddings(df)
visualize embeddings
print(df['embeddings'])

In [80]:
df = pd.DataFrame.from_dict(dataset)
df['embeddings']=df['embeddings'].apply(json.loads)

In [81]:
df.head()


,product_name,link,product_images,price,details,category,gender,embeddings
0,WORKOUT T-SHIRT,https://www.zara.com/in/en/workout-t-shirt-p05...,https://static.zara.net/photos///2023/I/0/2/p/...,32.06,T-shirt made of lightweight textured stretch f...,ACTIVEWEAR,men,"[0.00584839704, 0.00366304931, 0.0416106768, 0..."
1,SPORTS BACKPACK,https://www.zara.com/in/en/sports-backpack-p13...,https://static.zara.net/photos///2023/I/1/2/p/...,64.26,Multi-pocket backpack. Combination of material...,ACTIVEWEAR,men,"[0.00701390672, 0.00680825347, 0.0142998686, 0..."
2,COMPACT COTTON T-SHIRT,https://www.zara.com/in/en/compact-cotton-t-sh...,https://static.zara.net/photos///2023/I/0/2/p/...,32.06,Loose-fitting T-shirt made of compact cotton. ...,ACTIVEWEAR,men,"[0.0323305465, 0.0137906792, 0.0440658815, 0.0..."
3,TANK TOP WITH SLOGAN,https://www.zara.com/in/en/tank-top-with-sloga...,https://static.zara.net/photos///2023/I/0/2/p/...,27.86,Loose-fitting T-shirt made of cotton. Round ne...,ACTIVEWEAR,men,"[-0.0316199474, -0.0126086716, 0.036310371, 0...."
4,TRAINING SOCKS,https://www.zara.com/in/en/training-socks-p024...,https://static.zara.net/photos///2023/I/0/2/p/...,13.86,Pack of 2 pairs of seamless mesh socks with ri...,ACTIVEWEAR,men,"[0.0402002, -0.0139389914, 0.0203817375, 0.007..."


In [73]:
# Function to compress the embeddings without loosing / truncating the value
# {'errors': [{'message': "Document size limitation violated: number of elements an indexable Array (property 'embeddings') has (1408) exceeds maximum allowed (1000)", 'errorCode': 'SHRED_DOC_LIMIT_VIOLATION'}]}
"""
def compress_embeddings(embedding):
    compressed_embedding = zlib.compress(str(embedding).encode('utf-8'))
    return compressed_embedding

# Apply the function to the 'embeddings' column
df['embeddings'] = df['embeddings'].apply(compress_embeddings)
"""

In [82]:
# Define the maximum length
# to avoid error "{'errors': [{'message': "Document size limitation violated: number of elements an indexable Array (property 'embeddings') has (1408) exceeds maximum allowed (1000)", 'errorCode': 'SHRED_DOC_LIMIT_VIOLATION'}]}"

max_length = 1000

# Function to truncate lists to the maximum length
def truncate_list(lst, max_length):
    return lst[:max_length]

# Apply the function to the 'embeddings' column
df['embeddings'] = df['embeddings'].apply(lambda x: truncate_list(x, max_length))


In [83]:
df.head()


,product_name,link,product_images,price,details,category,gender,embeddings
0,WORKOUT T-SHIRT,https://www.zara.com/in/en/workout-t-shirt-p05...,https://static.zara.net/photos///2023/I/0/2/p/...,32.06,T-shirt made of lightweight textured stretch f...,ACTIVEWEAR,men,"[0.00584839704, 0.00366304931, 0.0416106768, 0..."
1,SPORTS BACKPACK,https://www.zara.com/in/en/sports-backpack-p13...,https://static.zara.net/photos///2023/I/1/2/p/...,64.26,Multi-pocket backpack. Combination of material...,ACTIVEWEAR,men,"[0.00701390672, 0.00680825347, 0.0142998686, 0..."
2,COMPACT COTTON T-SHIRT,https://www.zara.com/in/en/compact-cotton-t-sh...,https://static.zara.net/photos///2023/I/0/2/p/...,32.06,Loose-fitting T-shirt made of compact cotton. ...,ACTIVEWEAR,men,"[0.0323305465, 0.0137906792, 0.0440658815, 0.0..."
3,TANK TOP WITH SLOGAN,https://www.zara.com/in/en/tank-top-with-sloga...,https://static.zara.net/photos///2023/I/0/2/p/...,27.86,Loose-fitting T-shirt made of cotton. Round ne...,ACTIVEWEAR,men,"[-0.0316199474, -0.0126086716, 0.036310371, 0...."
4,TRAINING SOCKS,https://www.zara.com/in/en/training-socks-p024...,https://static.zara.net/photos///2023/I/0/2/p/...,13.86,Pack of 2 pairs of seamless mesh socks with ri...,ACTIVEWEAR,men,"[0.0402002, -0.0139389914, 0.0203817375, 0.007..."


In [61]:
from astrapy.db import AstraDB, AstraDBCollection

astra_db = AstraDB(token=os.getenv("ASTRA_DB_TOKEN"),api_endpoint=os.getenv("ASTRA_DB_ENDPOINT"))

In [62]:
collection = astra_db.create_collection(collection_name="fashion_buddy_workshop1",dimension=2000)

In [12]:
#Create embeddings

In [85]:
from ipywidgets import IntProgress
from IPython.display import display

def load_to_astra(df, collection):
  len_df=len(df)
  print(len_df)
  f=IntProgress(min=0,max=len_df)
  display(f)
  for i in range(len_df):
    f.value += 1
    f.description = str(f.value) + "/" + str(len_df)

    product_name=df.loc[i,"product_name"]
    link = df.loc[i,"link"]
    product_images = df.loc[i,"product_images"]
    price = df.loc[i,"price"]
    details = df.loc[i,"details"]
    category = df.loc[i,"category"]
    gender = df.loc[i,"gender"]
    embeddings = df.loc[i,"embeddings"]

    try:
      collection.insert_one({
          "_id":i,
          "product_name":product_name,
          "link":link,
         "product_images":product_images,
          "price":price,
          "details":details,
          "category":category,
          "gender": gender,
          "embeddings": embeddings})
    except Exception as error:
      error_info = json.loads(str(error))
      #print(error_info)

load_to_astra(df,collection)


936


IntProgress(value=0, max=936)

{'status': {'insertedIds': []}, 'errors': [{'message': 'Failed to insert document with _id 0: Document already exists with the given _id', 'errorCode': 'DOCUMENT_ALREADY_EXISTS'}]}
{'status': {'insertedIds': []}, 'errors': [{'message': 'Failed to insert document with _id 1: Document already exists with the given _id', 'errorCode': 'DOCUMENT_ALREADY_EXISTS'}]}
{'status': {'insertedIds': []}, 'errors': [{'message': 'Failed to insert document with _id 2: Document already exists with the given _id', 'errorCode': 'DOCUMENT_ALREADY_EXISTS'}]}
{'status': {'insertedIds': []}, 'errors': [{'message': 'Failed to insert document with _id 3: Document already exists with the given _id', 'errorCode': 'DOCUMENT_ALREADY_EXISTS'}]}
{'status': {'insertedIds': []}, 'errors': [{'message': 'Failed to insert document with _id 4: Document already exists with the given _id', 'errorCode': 'DOCUMENT_ALREADY_EXISTS'}]}
{'status': {'insertedIds': []}, 'errors': [{'message': 'Failed to insert document with _id 5: D

In [86]:
Takes an image fURL or image file) and returns it
def`get_imagelimage source):
if image_source. startswithl "http://") or image source,sta
img_bytes = requests.get(image_source).content
img = Image(img_bytes)
return img
else:
ing= Image. load_from_tilelimage_source)
return img
Gets image embeddings using Google Gemini model
def get_img_ embeddings(img, text=""):
embeddings = model.get_embeddings(
image=img.
contextual _text=text # passes text along with the image (multimodal)
return embeddings.image_embedding
Displays recommendations by category returned by Astra Vector Search
def show_recommendations(documents, category=None):
if category:
print()
print("Category:", category)
for. doc in documents:
print(doc["product_name"1, "$" + str(doc["price"]), ", Gender:". dec["gender")
print(doc["details"])
print[doc["Link"l)
print("Similarity score:; ", doc["$similarity"])
Finds the similar itens of the given reference image and categories (if any) aleng with a search prompt to give t
def find_similar_items(reference_image, categorles=None):
reference_img = get_image(reference_image)
print("Reference image:")
reference_img.shov()
print("Selected Categories:", ", ",joinfcategories) it categories else "None")
if categories:
for category in categories:

SyntaxError: unmatched ')' (<ipython-input-86-4d8e431c05a4>, line 1)